In [1]:
import kagglehub
from kagglehub import KaggleDatasetAdapter
import sys
import os
import pandas as pd
import backtrader as bt
import matplotlib as plt
import matplotlib.pyplot as plt
import seaborn as sns

b:\Projects\portfolio_management\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Download latest version
path = kagglehub.dataset_download("khalilvandian/portfolio-management")

# List all files and directories in the downloaded dataset path
files = os.listdir(path)
print("Files in dataset directory:", files)

Files in dataset directory: ['AVGO.csv', 'AXP.csv', 'BAC.csv', 'CB.csv', 'CMG.csv', 'EA.csv', 'EBAY.csv', 'GRMN.csv', 'IBM.csv', 'IT.csv', 'LEG.csv', 'MHK.csv', 'MS.csv', 'ORLY.csv', 'XL.csv']


In [3]:
def clean_stock_data(filepath):
    df = pd.read_csv(filepath)

    # Standardize column names (lowercase, no special characters)
    df.columns = [col.strip().lower().replace('/', '_').replace(' ', '_') for col in df.columns]

    # Rename 'close/last' to 'close' if needed
    if 'close_last' in df.columns:
        df.rename(columns={'close_last': 'close'}, inplace=True)

    # Remove dollar signs and convert to float
    df['close'] = df['close'].replace('[\\$,]', '', regex=True).astype(float)
    df['open'] = df['open'].replace('[\\$,]', '', regex=True).astype(float)
    df['high'] = df['high'].replace('[\\$,]', '', regex=True).astype(float)
    df['low'] = df['low'].replace('[\\$,]', '', regex=True).astype(float)

    # Parse dates
    df['date'] = pd.to_datetime(df['date'])
    df.set_index('date', inplace=True)
    df.sort_index(inplace=True)

    return df

In [4]:
stock_data_dict = {}
full_date_index = pd.date_range(start="2015-06-16", end="2025-06-13", freq='B')

for file in files: 
    ticker_name = file.replace(".csv", "").strip()
    file_path = os.path.join(path, file)

    stock_df = clean_stock_data(file_path)
    stock_df = stock_df.reindex(full_date_index)
    stock_df.ffill(inplace=True) 
    stock_df.fillna(0, inplace=True)

    stock_data_dict[ticker_name] = stock_df


# Add no risk option
# copy 1st df and change all values to a constant value
no_risk = stock_data_dict["AVGO"].copy()
no_risk["close"] = 1
no_risk["open"] = 1
no_risk["high"] = 1
no_risk["low"] = 1
no_risk["volume"] = 0
stock_data_dict["NoRisk"] = no_risk

tickers = list(stock_data_dict.keys())
print(tickers)

['AVGO', 'AXP', 'BAC', 'CB', 'CMG', 'EA', 'EBAY', 'GRMN', 'IBM', 'IT', 'LEG', 'MHK', 'MS', 'ORLY', 'XL', 'NoRisk']


In [5]:
# Find and print tickers with any NaN in their "close" column
for ticker, df in stock_data_dict.items():
	if df["close"].isna().any():
		print(f"{ticker} has NaN values in 'close':")
		print(df[df["close"].isna()])

In [6]:
no_risk

,close,volume,open,high,low
2015-06-16,1,0,1,1,1
2015-06-17,1,0,1,1,1
2015-06-18,1,0,1,1,1
2015-06-19,1,0,1,1,1
2015-06-22,1,0,1,1,1
...,...,...,...,...,...
2025-06-09,1,0,1,1,1
2025-06-10,1,0,1,1,1
2025-06-11,1,0,1,1,1
2025-06-12,1,0,1,1,1


# Backtrade Functions

In [7]:
class BuyAndHoldCustom(bt.Strategy):

    # Add a parameters class to accept allocations
    params = (('allocations', {}),)

    def __init__(self):
        self.bought = {}
        # Access the allocations from the parameters
        self.allocations = self.p.allocations

    def next(self):
        for data in self.datas:
            name = data._name
            if not self.getposition(data).size and name not in self.bought:
                cash = self.broker.get_cash()
                allocation = self.params.allocations[name]
                size = int(allocation / data.close[0])
                self.buy(data=data, size=size)
                self.bought[name] = True

    def stop(self):
        # Called at the end of the backtest
        for data in self.datas:
            position = self.getposition(data)
            if position.size > 0:
                self.sell(data=data, size=position.size)
                # print(f"SELL at END: {data._name} @ {data.close[0]:.2f}, Size: {position.size}")

def run_custom_backtest(dataframes_dict, allocations_dict, start_date, end_date, budget):
    cerebro = bt.Cerebro()
    cerebro.broker.set_cash(budget)

    filtered_dataframes = {}
    for stock_name, df in dataframes_dict.items():
        df = df.copy()
        # df['Date'] = pd.to_datetime(df['date'])
        # df.set_index('Date', inplace=True)
        df = df.loc[start_date:end_date]
        df.columns = df.columns.str.capitalize()  # Ensure 'Close' column exists
        filtered_dataframes[stock_name] = df
        feed = bt.feeds.PandasData(dataname=df)
        cerebro.adddata(feed, name=stock_name)

    # Pass allocations into strategy as parameter
    cerebro.addstrategy(BuyAndHoldCustom, allocations=allocations_dict)

    start_value = round(cerebro.broker.getvalue(), 2)
    cerebro.run()
    end_value = round(cerebro.broker.getvalue(), 2)

    return {
        'start_value': round(start_value, 2),
        'end_value': round(end_value, 2),
        'total_gain': round(end_value - start_value, 2),
        'percent_gain': round((end_value - start_value) / (start_value) * 100, 2),
        'cerebro': cerebro
    }


# Strategy Constant Weights

In [8]:
all_results = {
    "Method Name": "test",
    "Investment Period (Months)": int(1),
    "Initial Value": 100000.00,
    "Final Value": 96057.77,
    "Total Gain": -3942.23,
    "Percent Gain": "-3.94 %"
}

periods = ['2025-01-01', '2025-02-01', '2025-03-01', '2025-04-01', '2025-05-01', '2025-06-01']
all_results = pd.DataFrame([all_results])
all_results["Investment Period (Months)"] = all_results["Investment Period (Months)"].astype(int)
all_results

,Method Name,Investment Period (Months),Initial Value,Final Value,Total Gain,Percent Gain
0,test,1,100000.0,96057.77,-3942.23,-3.94 %


In [9]:
# create dataframe for the investment
investment_strategy = pd.DataFrame(tickers, columns=["Symbol"])

# set budget as 1 dollar
budget = 100000

investment_strategy["Weight"] = round(1/len(investment_strategy), 3)
investment_strategy["Position_Size"] = budget * investment_strategy["Weight"]

investment_strategy

,Symbol,Weight,Position_Size
0,AVGO,0.062,6200.0
1,AXP,0.062,6200.0
2,BAC,0.062,6200.0
3,CB,0.062,6200.0
4,CMG,0.062,6200.0
5,EA,0.062,6200.0
6,EBAY,0.062,6200.0
7,GRMN,0.062,6200.0
8,IBM,0.062,6200.0
9,IT,0.062,6200.0


In [10]:
strategy_one_path = "./Data/First Strategy"
strategy_one_file_name = "constant_weights_portfolio.csv"
pd.read_csv(os.path.join(strategy_one_path, strategy_one_file_name))

,Symbol,Weight,Position_Size
0,AVGO,0.062,6200.0
1,AXP,0.062,6200.0
2,BAC,0.062,6200.0
3,CB,0.062,6200.0
4,CMG,0.062,6200.0
5,EA,0.062,6200.0
6,EBAY,0.062,6200.0
7,GRMN,0.062,6200.0
8,IBM,0.062,6200.0
9,IT,0.062,6200.0


In [11]:
position_size_dict = {}
for index, row in investment_strategy.iterrows():
    position_size_dict[row['Symbol']] = row['Position_Size']

# create company dicts
dataframes_dict = stock_data_dict.copy()


for period in periods:
    result = run_custom_backtest(
        dataframes_dict=dataframes_dict,
        allocations_dict=position_size_dict,
        start_date='2024-12-01',
        end_date=period,
        budget=budget
    )

    # Fill in all columns: Method Name, Investment Period (Months), Initial Value, Final Value, Total Gain, Percent Gain
    method_name = "Constant Weights"
    # Calculate months between start and end
    investment_months = (pd.to_datetime(period, format='%Y-%m-%d') - pd.to_datetime('2024-12-31', format='%Y-%m-%d')).days // 30 + 1
    values = [
        method_name,
        investment_months,
        result['start_value'],
        result['end_value'],
        result['total_gain'],
        f"{result['percent_gain']} %"
    ]
    
    all_results.loc[len(all_results), :] = values

all_results

,Method Name,Investment Period (Months),Initial Value,Final Value,Total Gain,Percent Gain
0,test,1.0,100000.0,96057.77,-3942.23,-3.94 %
1,Constant Weights,1.0,100000.0,95459.33,-4540.67,-4.54 %
2,Constant Weights,2.0,100000.0,99755.73,-244.27,-0.24 %
3,Constant Weights,3.0,100000.0,97984.05,-2015.95,-2.02 %
4,Constant Weights,4.0,100000.0,93833.19,-6166.81,-6.17 %
5,Constant Weights,5.0,100000.0,92520.19,-7479.81,-7.48 %
6,Constant Weights,6.0,100000.0,96057.77,-3942.23,-3.94 %


In [12]:
# Generate plot
cerebro = result["cerebro"]
# cerebro.plot(numfigs=16)

try:
	figs = cerebro.plot(style="candlestick")  # You can choose style: line, candlestick, ohlc

	# Set large size and save the first figure
	fig = figs[0][0]  # First subplot in first figure
	fig.set_size_inches(50, 50)  # Width, Height in inches
	fig.savefig("Results/constant_weights_backtest.png", dpi=100)  # Save with high resolution
except ValueError as e:
	print("Plotting failed due to:", e)
	print("Try removing tickers with NaN values from your data.")

<IPython.core.display.Javascript object>

# Strategy Morkowitz

## Final Portfolio

In [13]:
strategy_two_path = "Data/Second Strategy"
final_portfolio_name = "final_portfolio.csv"

final_portfolio = pd.read_csv(os.path.join(strategy_two_path, final_portfolio_name)).round(decimals=3)
final_portfolio

,Ticker,Weight
0,NoRisk,0.250
1,CB,0.101
2,ORLY,0.095
3,IBM,0.093
4,GRMN,0.085
5,EA,0.065
6,AXP,0.054
7,EBAY,0.047
8,MS,0.046
9,BAC,0.046


In [14]:
# create dataframe for the investment
investment_strategy = final_portfolio.rename(columns={"Ticker":"Symbol"}).copy()

# set budget as 1 dollar
budget = 100000

# investment_strategy["Weight"] = round(1/len(investment_strategy), 3)
investment_strategy["Position_Size"] = budget * investment_strategy["Weight"]
investment_strategy.loc[len(investment_strategy)] = ["NoRisk", 0, 0]

investment_strategy

,Symbol,Weight,Position_Size
0,NoRisk,0.250,25000.0
1,CB,0.101,10100.0
2,ORLY,0.095,9500.0
3,IBM,0.093,9300.0
4,GRMN,0.085,8500.0
5,EA,0.065,6500.0
6,AXP,0.054,5400.0
7,EBAY,0.047,4700.0
8,MS,0.046,4600.0
9,BAC,0.046,4600.0


In [15]:
position_size_dict = {}
for index, row in investment_strategy.iterrows():
    position_size_dict[row['Symbol']] = row['Position_Size']

# create company dicts
dataframes_dict = stock_data_dict.copy()


for period in periods:
    result = run_custom_backtest(
        dataframes_dict=dataframes_dict,
        allocations_dict=position_size_dict,
        start_date='2024-12-01',
        end_date=period,
        budget=budget
    )

    # Fill in all columns: Method Name, Investment Period (Months), Initial Value, Final Value, Total Gain, Percent Gain
    method_name = "Morkowitz Final Portfolio"
    # Calculate months between start and end
    investment_months = (pd.to_datetime(period, format='%Y-%m-%d') - pd.to_datetime('2024-12-31', format='%Y-%m-%d')).days // 30 + 1
    values = [
        method_name,
        investment_months,
        result['start_value'],
        result['end_value'],
        result['total_gain'],
        f"{result['percent_gain']} %"
    ]
    
    all_results.loc[len(all_results), :] = values

all_results

,Method Name,Investment Period (Months),Initial Value,Final Value,Total Gain,Percent Gain
0,test,1.0,100000.0,96057.77,-3942.23,-3.94 %
1,Constant Weights,1.0,100000.0,95459.33,-4540.67,-4.54 %
2,Constant Weights,2.0,100000.0,99755.73,-244.27,-0.24 %
3,Constant Weights,3.0,100000.0,97984.05,-2015.95,-2.02 %
4,Constant Weights,4.0,100000.0,93833.19,-6166.81,-6.17 %
5,Constant Weights,5.0,100000.0,92520.19,-7479.81,-7.48 %
6,Constant Weights,6.0,100000.0,96057.77,-3942.23,-3.94 %
7,Morkowitz Final Portfolio,1.0,100000.0,95956.62,-4043.38,-4.04 %
8,Morkowitz Final Portfolio,2.0,100000.0,99527.87,-472.13,-0.47 %
9,Morkowitz Final Portfolio,3.0,100000.0,99646.60,-353.40,-0.35 %


## Max Sharpe Portfolio

In [16]:
strategy_two_path = "./Data/Second Strategy"
max_sharpe_portfolio_file_name = "max_sharpe_portfolio.csv"

max_sharpe_portfolio = pd.read_csv(os.path.join(strategy_two_path, max_sharpe_portfolio_file_name)).round(decimals=3)
max_sharpe_portfolio.loc[max_sharpe_portfolio["Ticker"] == "RISK_FREE", "Ticker"] = "NoRisk"

# create dataframe for the investment
investment_strategy = max_sharpe_portfolio.rename(columns={"Ticker":"Symbol"}).copy()

# set budget as 1 dollar
budget = 100000

# investment_strategy["Weight"] = round(1/len(investment_strategy), 3)
investment_strategy["Position_Size"] = budget * investment_strategy["Weight"]

investment_strategy

,Symbol,Weight,Position_Size
0,LEG,0.737,73700.0
1,XL,0.263,26300.0
2,MHK,0.000,0.0
3,AXP,0.000,0.0
4,IBM,0.000,0.0
5,MS,0.000,0.0
6,IT,0.000,0.0
7,BAC,0.000,0.0
8,CMG,0.000,0.0
9,GRMN,0.000,0.0


In [17]:
position_size_dict = {}
for index, row in investment_strategy.iterrows():
    position_size_dict[row['Symbol']] = row['Position_Size']

# create company dicts
dataframes_dict = stock_data_dict.copy()


for period in periods:
    result = run_custom_backtest(
        dataframes_dict=dataframes_dict,
        allocations_dict=position_size_dict,
        start_date='2024-12-01',
        end_date=period,
        budget=budget
    )

    # Fill in all columns: Method Name, Investment Period (Months), Initial Value, Final Value, Total Gain, Percent Gain
    method_name = "Max Sharpe Portfolio"
    # Calculate months between start and end
    investment_months = (pd.to_datetime(period, format='%Y-%m-%d') - pd.to_datetime('2024-12-31', format='%Y-%m-%d')).days // 30 + 1
    values = [
        method_name,
        investment_months,
        result['start_value'],
        result['end_value'],
        result['total_gain'],
        f"{result['percent_gain']} %"
    ]
    
    all_results.loc[len(all_results), :] = values

all_results

,Method Name,Investment Period (Months),Initial Value,Final Value,Total Gain,Percent Gain
0,test,1.0,100000.0,96057.77,-3942.23,-3.94 %
1,Constant Weights,1.0,100000.0,95459.33,-4540.67,-4.54 %
2,Constant Weights,2.0,100000.0,99755.73,-244.27,-0.24 %
3,Constant Weights,3.0,100000.0,97984.05,-2015.95,-2.02 %
4,Constant Weights,4.0,100000.0,93833.19,-6166.81,-6.17 %
5,Constant Weights,5.0,100000.0,92520.19,-7479.81,-7.48 %
6,Constant Weights,6.0,100000.0,96057.77,-3942.23,-3.94 %
7,Morkowitz Final Portfolio,1.0,100000.0,95956.62,-4043.38,-4.04 %
8,Morkowitz Final Portfolio,2.0,100000.0,99527.87,-472.13,-0.47 %
9,Morkowitz Final Portfolio,3.0,100000.0,99646.60,-353.40,-0.35 %


In [18]:
# Generate plot
cerebro = result["cerebro"]
# cerebro.plot(numfigs=16)

try:
	figs = cerebro.plot()  # You can choose style: line, candlestick, ohlc

	# Set large size and save the first figure
	fig = figs[0][0]  # First subplot in first figure
	fig.set_size_inches(24, 60)  # Width, Height in inches
	fig.savefig("large_backtest_plot.png", dpi=100)  # Save with high resolution
except ValueError as e:
	print("Plotting failed due to:", e)
	print("Try removing tickers with NaN values from your data.")

<IPython.core.display.Javascript object>

## Min Variance

In [19]:
strategy_two_path = "./Data/Second Strategy"
min_variance_portfolio_file_name = "min_variance_portfolio.csv"

min_variance_portfolio = pd.read_csv(os.path.join(strategy_two_path, min_variance_portfolio_file_name)).round(decimals=3)
min_variance_portfolio.loc[min_variance_portfolio["Ticker"] == "RISK_FREE", "Ticker"] = "NoRisk"

# create dataframe for the investment
investment_strategy = min_variance_portfolio.rename(columns={"Ticker":"Symbol"}).copy()

# set budget as 1 dollar
budget = 100000

# investment_strategy["Weight"] = round(1/len(investment_strategy), 3)
investment_strategy["Position_Size"] = budget * investment_strategy["Weight"]

investment_strategy

,Symbol,Weight,Position_Size
0,NoRisk,0.552,55200.0
1,IBM,0.094,9400.0
2,EA,0.079,7900.0
3,LEG,0.074,7400.0
4,CB,0.058,5800.0
5,EBAY,0.045,4500.0
6,CMG,0.028,2800.0
7,ORLY,0.022,2200.0
8,XL,0.020,2000.0
9,GRMN,0.020,2000.0


In [20]:
position_size_dict = {}
for index, row in investment_strategy.iterrows():
    position_size_dict[row['Symbol']] = row['Position_Size']

# create company dicts
dataframes_dict = stock_data_dict.copy()


for period in periods:
    result = run_custom_backtest(
        dataframes_dict=dataframes_dict,
        allocations_dict=position_size_dict,
        start_date='2024-12-01',
        end_date=period,
        budget=budget
    )

    # Fill in all columns: Method Name, Investment Period (Months), Initial Value, Final Value, Total Gain, Percent Gain
    method_name = "Min Variance Portfolio"
    # Calculate months between start and end
    investment_months = (pd.to_datetime(period, format='%Y-%m-%d') - pd.to_datetime('2024-12-31', format='%Y-%m-%d')).days // 30 + 1
    values = [
        method_name,
        investment_months,
        result['start_value'],
        result['end_value'],
        result['total_gain'],
        f"{result['percent_gain']} %"
    ]
    
    all_results.loc[len(all_results), :] = values

all_results = all_results.iloc[1:]
all_results

,Method Name,Investment Period (Months),Initial Value,Final Value,Total Gain,Percent Gain
1,Constant Weights,1.0,100000.0,95459.33,-4540.67,-4.54 %
2,Constant Weights,2.0,100000.0,99755.73,-244.27,-0.24 %
3,Constant Weights,3.0,100000.0,97984.05,-2015.95,-2.02 %
4,Constant Weights,4.0,100000.0,93833.19,-6166.81,-6.17 %
5,Constant Weights,5.0,100000.0,92520.19,-7479.81,-7.48 %
6,Constant Weights,6.0,100000.0,96057.77,-3942.23,-3.94 %
7,Morkowitz Final Portfolio,1.0,100000.0,95956.62,-4043.38,-4.04 %
8,Morkowitz Final Portfolio,2.0,100000.0,99527.87,-472.13,-0.47 %
9,Morkowitz Final Portfolio,3.0,100000.0,99646.60,-353.40,-0.35 %
10,Morkowitz Final Portfolio,4.0,100000.0,98315.23,-1684.77,-1.68 %


# Strategy LSTM

In [21]:
lstm_res = pd.read_csv("Results/Sergio Results.csv")

# Combine Results and Plot

In [22]:
final_results = pd.concat([all_results, lstm_res])
final_results

,Method Name,Investment Period (Months),Initial Value,Final Value,Total Gain,Percent Gain
1,Constant Weights,1.0,100000.0,95459.33,-4540.67,-4.54 %
2,Constant Weights,2.0,100000.0,99755.73,-244.27,-0.24 %
3,Constant Weights,3.0,100000.0,97984.05,-2015.95,-2.02 %
4,Constant Weights,4.0,100000.0,93833.19,-6166.81,-6.17 %
5,Constant Weights,5.0,100000.0,92520.19,-7479.81,-7.48 %
6,Constant Weights,6.0,100000.0,96057.77,-3942.23,-3.94 %
7,Morkowitz Final Portfolio,1.0,100000.0,95956.62,-4043.38,-4.04 %
8,Morkowitz Final Portfolio,2.0,100000.0,99527.87,-472.13,-0.47 %
9,Morkowitz Final Portfolio,3.0,100000.0,99646.60,-353.40,-0.35 %
10,Morkowitz Final Portfolio,4.0,100000.0,98315.23,-1684.77,-1.68 %


In [ ]:
df = final_results.copy()
df['Percent Gain'] = df['Percent Gain'].str.replace('%', '').str.strip().astype(float)

# Define custom markers for each strategy
unique_strategies = df["Method Name"].unique()
marker_styles = ['o', 's', 'D', '^', 'X', 'P', '*', 'v', '<', '>', 'H']
strategy_marker_map = {strategy: marker_styles[i % len(marker_styles)] for i, strategy in enumerate(unique_strategies)}

# Create the plot
plt.figure(figsize=(14, 7))
ax = sns.pointplot(
    data=df,
    x="Investment Period (Months)",
    y="Percent Gain",
    hue="Method Name",
    dodge=0.3,
    errorbar='sd',
    linestyles='-'
)

# Customize line style and marker for each strategy
handles, labels = ax.get_legend_handles_labels()
for line, label in zip(ax.lines, labels):
    line.set_marker(strategy_marker_map[label])
    if label == "Constant Weights":
        line.set_linestyle("--")
        line.set_color("black")

# Final formatting
plt.title("Strategy Comparison by Investment Period", fontsize=16)
plt.xlabel("Investment Period (Months)", fontsize=12)
plt.ylabel("Percent Gain", fontsize=12)
plt.xticks(rotation=45)
plt.grid(True)
plt.legend(title="Strategy", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()

# Save the plot
output_path = "Results/strategy_comparison_with_markers.png"
plt.savefig(output_path, dpi=300)

output_path

'Results/strategy_comparison_with_markers.png'